In [1]:
%matplotlib inline

import warnings
from text_unidecode import unidecode
from collections import deque
warnings.filterwarnings('ignore')


import pandas as pd
from sklearn.manifold import TSNE
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
from node2vec import Node2Vec

sns.set_style('whitegrid')

In [2]:
data = pd.read_csv('KNN_result.csv')

In [8]:
data

,PANEL_ID,APP_CATEGORY,APP_USE_TME,CATEGORY_CODE,Area_Name2,PAY_CNT,PAY_MEAN_AMOUNT,gender,age,loctown,...,education,job,family_cnt,has_baby,has_student,has_silver,ann_income,home_cat,home_pay,KNN_LABEL
0,A0000910,LIFESTYLE,932016,701,화성시,38,98782,1,5,1,...,4,6,5,0,0,0,4,2,1,0
1,A0001778,SOCIAL,185520,501,강동구,278,271246,2,4,1,...,4,997,4,0,0,0,4,2,1,0
2,A0005079,GAME,270570,401,강서구,23,20009,1,5,1,...,6,5,1,0,0,0,6,5,1,2
3,A0005444,EDUCATION,441035,502,동대문구,237,195353,2,4,1,...,2,11,4,0,0,0,5,2,1,0
4,A0009694,ENTERTAINMENT,539364,201,강서구,287,71854,2,4,1,...,4,11,4,0,0,0,4,2,2,0
5,H0000102,SOCIAL,113310,1799,송파구,72,103809,2,2,1,...,2,8,3,0,0,0,4,5,2,3
6,H0000408,EDUCATION,206050,1799,의왕시,3,37133,1,2,1,...,3,12,4,0,0,0,3,2,1,3
7,H0000522,LIFESTYLE,93952,101,수원시영통구,130,202345,1,3,1,...,6,7,5,0,0,0,6,1,1,3
8,H0000617,EDUCATION,537055,501,용인시수지구,290,960340,2,3,1,...,4,11,2,0,0,0,4,2,1,0
9,H0001082,SOCIAL,132152,101,수원시영통구,18,11675,2,2,1,...,4,13,5,0,0,1,3,2,1,3


In [3]:
pd.concat([data['PANEL_ID'],pd.get_dummies(data['KNN_LABEL'])],axis=1)

,PANEL_ID,0,1,2,3,4
0,A0000910,1,0,0,0,0
1,A0001778,1,0,0,0,0
2,A0005079,0,0,1,0,0
3,A0005444,1,0,0,0,0
4,A0009694,1,0,0,0,0
5,H0000102,0,0,0,1,0
6,H0000408,0,0,0,1,0
7,H0000522,0,0,0,1,0
8,H0000617,1,0,0,0,0
9,H0001082,0,0,0,1,0


In [17]:
pd.get_dummies(data['KNN_LABEL']).values

array([[1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0],
       ...,
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0]], dtype=uint8)

In [13]:
pd.get_dummies(data['KNN_LABEL']).shape

(7667, 5)

In [18]:
pd.get_dummies(data['KNN_LABEL']).values.T

array([[1, 1, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [5]:
wmatrix = pd.DataFrame(np.matmul(pd.get_dummies(data['KNN_LABEL']).values,pd.get_dummies(data['KNN_LABEL']).values.T),columns=data['PANEL_ID'],index=data['PANEL_ID'])

In [7]:
np.fill_diagonal(tmp,0)

In [ ]:
tmp = wmatrix.values

In [8]:
tmpdf = pd.DataFrame(tmp,columns=data['PANEL_ID'],index=data['PANEL_ID'])

In [9]:
tt = tmpdf.apply(lambda x : np.array(x[x==1].index),axis=1)

In [49]:
tt

PANEL_ID
A0000910    [A0001778, A0005444, A0009694, H0000617, H0002...
A0001778    [A0000910, A0005444, A0009694, H0000617, H0002...
A0005079    [H0003035, H0007473, H0010097, H0011933, H0012...
A0005444    [A0000910, A0001778, A0009694, H0000617, H0002...
A0009694    [A0000910, A0001778, A0005444, H0000617, H0002...
H0000102    [H0000408, H0000522, H0001082, H0001454, H0001...
H0000408    [H0000102, H0000522, H0001082, H0001454, H0001...
H0000522    [H0000102, H0000408, H0001082, H0001454, H0001...
H0000617    [A0000910, A0001778, A0005444, A0009694, H0002...
H0001082    [H0000102, H0000408, H0000522, H0001454, H0001...
H0001454    [H0000102, H0000408, H0000522, H0001082, H0001...
H0001977    [H0000102, H0000408, H0000522, H0001082, H0001...
H0001978    [H0000102, H0000408, H0000522, H0001082, H0001...
H0002383    [A0000910, A0001778, A0005444, A0009694, H0000...
H0002740    [A0000910, A0001778, A0005444, A0009694, H0000...
H0002821    [H0000102, H0000408, H0000522, H0001082, H0001...

In [14]:
del tt,tmpdf,tmp,wmatrix

In [38]:
tmpdf.iloc[0,:][tmpdf.iloc[0,:]==1].index

Index(['A0001778', 'A0005444', 'A0009694', 'H0000617', 'H0002383', 'H0002740',
       'H0004053', 'H0006332', 'H0006940', 'H0008238',
       ...
       'P0309645', 'P0309783', 'P0309852', 'P0309893', 'P0310070', 'S0002210',
       'S0002264', 'S0003875', 'S0004212', 'T0012282'],
      dtype='object', name='PANEL_ID', length=3194)

In [11]:
%matplotlib inline

import warnings
from text_unidecode import unidecode
from collections import deque
warnings.filterwarnings('ignore')


import pandas as pd
from sklearn.manifold import TSNE
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
from node2vec import Node2Vec

sns.set_style('whitegrid')

In [12]:
graph = nx.Graph()

In [13]:
for i,j in  zip(data['PANEL_ID'],tt):
    for k in j:
        graph.add_edge(i,k)

In [15]:
node2vec = Node2Vec(graph, dimensions=5, walk_length=16, num_walks=100, workers=2)

Computing transition probabilities:   0%|                                        | 19/7667 [23:45<156:59:20, 73.90s/it]

KeyboardInterrupt: 

In [ ]:
model = node2vec.fit(window=10, min_count=1)

In [ ]:
panel_nodes = [x for x in model.wv.vocab if len(x) > 3]
embeddings = np.array([model.wv[x] for x in panel_nodes])

In [ ]:
tsne = TSNE(n_components=2, random_state=7, perplexity=15)
embeddings_2d = tsne.fit_transform(embeddings)

In [ ]:
figure = plt.figure(figsize=(11, 9))

ax = figure.add_subplot(111)

ax.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1])#, c=color)